In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
trian_csv=pd.read_csv('../input/histopathologic-cancer-detection/train_labels.csv')

In [ ]:
trian_csv.head()

In [ ]:
trian_csv.isnull().sum()

In [ ]:
sns.countplot(x='label',data=trian_csv)

In [ ]:
trian_csv['label'] = trian_csv['label'].astype('str')

In [ ]:
trian_csv["id"] = trian_csv["id"].apply(lambda x: x + ".tif")
trian_csv.head()

In [ ]:
import tensorflow as tf
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.0,
            rotation_range=10,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            validation_split = 0.1
            )

In [ ]:
train_generator = datagen.flow_from_dataframe(
            dataframe=trian_csv,
            directory="../input/histopathologic-cancer-detection/train/",
            x_col="id",
            y_col="label",
            subset="training",
            batch_size=128,
            seed=42,
            shuffle=True,
            class_mode="binary",
            target_size=(224,224))

In [ ]:
classes=list(train_generator.class_indices.keys())

In [ ]:
classes

In [ ]:
classes=list(train_generator.class_indices.keys())
plt.figure(figsize=(20,20))
for X_batch, y_batch in train_generator:
    # create a grid of 3x3 images
    for i in range(0,36):
        plt.subplot(6,6,i+1)
        plt.imshow(X_batch[i])
    # show the plot
    plt.show()
    break

In [ ]:
datagen_valid = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.,
            validation_split = 0.1
            )

In [ ]:
valid_generator = datagen_valid.flow_from_dataframe(
            dataframe=trian_csv,
            directory="../input/histopathologic-cancer-detection/train",
            x_col="id",
            y_col="label",
            subset="validation",
            batch_size=128,
            seed=42,
            shuffle=True,
            class_mode="binary",
            target_size=(224,224))

In [ ]:
plt.figure(figsize=(20,20))
for X_batch, y_batch in valid_generator:
    # create a grid of 3x3 images
    for i in range(0,36):
        plt.subplot(6,6,i+1)
        plt.imshow(X_batch[i])
    # show the plot
    plt.show()
    break

In [ ]:
X_batch.shape

In [ ]:
test_csv=pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')

In [ ]:
test_csv.isnull().sum()

In [ ]:
sns.countplot(x='label',data=test_csv)

In [ ]:
test_csv['label'] = test_csv['label'].astype('str')

In [ ]:
test_csv["id"] = test_csv["id"].apply(lambda x: x + ".tif")
test_csv.head()

In [ ]:
datagen_test = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.
            )

In [ ]:
test_generator = datagen_test.flow_from_dataframe(
            dataframe=test_csv,
            directory="../input/histopathologic-cancer-detection/test",
            x_col="id",
            y_col="label",
            batch_size=128,
            seed=42,
            shuffle=True,
            target_size=(224,224))

In [ ]:
plt.figure(figsize=(20,20))
for X_batch, y_batch in valid_generator:
    # create a grid of 3x3 images
    for i in range(0,36):
        plt.subplot(6,6,i+1)
        plt.imshow(X_batch[i])
    # show the plot
    plt.show()
    break

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense ,Flatten ,Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization ,GlobalMaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau ,ModelCheckpoint

In [ ]:
#optimizer
optimizer_adam=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.99)

In [ ]:
#callbacks

EarlyStop=EarlyStopping(patience=10,restore_best_weights=True)
Reduce_LR=ReduceLROnPlateau(monitor='val_accuracy',verbose=2,factor=0.5,min_lr=0.00001)
model_check=ModelCheckpoint('model.hdf5',monitor='val_loss',verbose=1,save_best_only=True)
callback=[EarlyStop , Reduce_LR,model_check]

In [ ]:
model=Sequential([
                  Conv2D(64,3,activation='relu',kernel_initializer='he_normal',input_shape=(224,224,3)),
                  BatchNormalization(),
                  Conv2D(128,3,activation='relu',kernel_initializer='he_normal'),
                  BatchNormalization(),
                  MaxPooling2D(3),
                  Conv2D(256,3,activation='relu'),
                  BatchNormalization(),
                  MaxPooling2D(3),
                  Conv2D(512,3,padding='valid',activation='relu'),
                  BatchNormalization(),
                  GlobalMaxPool2D(),
                  Flatten(),
                  Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                  BatchNormalization(),
                  Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                  BatchNormalization(),
                  Dense(1,activation='sigmoid',kernel_initializer='glorot_normal')
                  
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,validation_data=valid_generator,epochs=50,batch_size=128,
                  steps_per_epoch=len(train_generator)//128,validation_steps=len(valid_generator)//128,
                  callbacks=callback, verbose=1)

In [ ]:
model.evaluate(test_generator)

In [ ]:
#plotting training values
import seaborn as sns
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='green', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()